In [1]:
include("build_profiles.jl")

build_ngram_profiles (generic function with 1 method)

In [2]:
@time build_ngram_profiles(LANGS, "profiles")

# ara
total vocab: 379.986k; tokens: 0.548987M;

 dataset ratio: [0.40910440502234113, 0.5908955949776589](vocab) Float32[0.4240475, 0.5759525](token)
Profile size: 28.078k(7.0%) @freq=111.0; [min(28078, 121357)]


 18.364420 seconds (54.41 M allocations: 3.977 GiB, 14.22% gc time, 10.84% compilation time)


# bel
total vocab: 338.556k; tokens: 0.439762M;

 dataset ratio: [0.27132630832131927, 0.7286736916786808](vocab) Float32[0.18398176, 0.8160182](token)
Profile size: 21.333k(6.0%) @freq=125.0; [min(21333, 60077)]
 16.392093 seconds (44.05 M allocations: 3.338 GiB, 15.24% gc time)


...

 dataset ratio: [0.2853411172367744, 0.7146588827632255](vocab) Float32[0.19540617, 0.80459386](token)
Profile size: 10.048k(6.0%) @freq=161.0; [min(10048, 42022)]
 13.573327 seconds (42.68 M allocations: 3.250 GiB, 17.35% gc time)


# zho
total vocab: 5313.111k; tokens: 5.939956M;

 dataset ratio: [0.3429495100637109, 0.6570504899362891](vocab) Float32[0.45268676, 0.5473132](token)
Profile size: 100.0k(2.0%) @freq=29.0; [min(1897912, 277102)]


 44.031133 seconds (46.57 M allocations: 4.540 GiB, 21.56% gc time)


2564.095682 seconds (5.99 G allocations: 719.891 GiB, 19.03% gc time, 0.09% compilation time)
